In [9]:
# %load_ext autoreload
# %autoreload 2

In [18]:
import syft as sy
import numpy as np
import pandas as pd

In [2]:
domain = sy.login(
    email="info@openmined.org",
    password="changethis",
    port=8081,

)

Connecting to http://localhost:8081... done! 	 Logging into mednode... done!


In [4]:
# Let's create an empty dataset

domain.create_dataset(
    name="Empty Dataset",
    description="Test delete",
)

.

WARNING - Dataset Name Conflict: A dataset named 'Empty Dataset' already exists.

Do you want to upload this dataset anyway? (y/n)y

Creating an empty dataset... Creating... SUCCESS!

In [10]:
# We can see that there are not existing assets attached to the dataset
domain.datasets

Idx,Name,Description,Assets,Id
[0],Empty Dataset,Test delete,,17030881-6f2a-45e0-8283-434fdd014740


In [15]:
# Let's grab the dataset pointer
dataset_pointer = domain.datasets[0]

In [16]:
dataset_pointer

Dataset: Empty Dataset
Description: Test delete



Asset Key,Type,Shape


In [23]:
# Let's add a random asset to the dataset

dataset_pointer.add(name="random-asset", value=np.random.rand(100, 8))



WARNING - Non-DP Asset: You just passed in a asset 'random-asset' which cannot be tracked with differential privacy because it is a <class 'numpy.ndarray'> object.

This means you'll need to manually approve any requests which leverage this data. If this is ok with you, proceed. If you'd like to use automatic differential privacy budgeting, please pass in a DP-compatible tensor type such as by calling .private() on a sy.Tensor with a np.int32 or np.float32 inside.
Are you sure you want to proceed? (y/n)y
Loading dataset... uploading... SUCCESS!

In [24]:
# Let's check the dataset pointer
dataset_pointer

Dataset: Empty Dataset
Description: Test delete



Asset Key,Type,Shape
"[""random-asset""]",ndarray,"(100, 8)"


In [25]:
# Great !!! we have an asset attached to the dataset
# Let's try an asset with the same name again

dataset_pointer.add(name="random-asset", value=np.random.rand(50, 4))



WARNING - Non-DP Asset: You just passed in a asset 'random-asset' which cannot be tracked with differential privacy because it is a <class 'numpy.ndarray'> object.

This means you'll need to manually approve any requests which leverage this data. If this is ok with you, proceed. If you'd like to use automatic differential privacy budgeting, please pass in a DP-compatible tensor type such as by calling .private() on a sy.Tensor with a np.int32 or np.float32 inside.
Are you sure you want to proceed? (y/n)y


KeyError: 'Asset with name: `random-asset` already exists.Please use a different name.'

In [26]:
# Great!!! So, we cannot add asset with the same name.
# Let's try and delete the asset

dataset_pointer.delete(name="random-asset")

True

In [27]:
# The dataset is deleted. 
# let's check the dataset_pointer again

dataset_pointer

Dataset: Empty Dataset
Description: Test delete



Asset Key,Type,Shape


In [28]:
# We don't have any assets attached, which is what is expected.

# If we delete an asset that does not exists, we will get an error
dataset_pointer.delete("key-not-exists")

KeyError: 'The asset with name `key-not-exists` does not exists.'

In [29]:
# Great, now that we can add/delete assets, we can upload the dataset
# in batches, let's try to do that.

Let's load some data... into memory

In [33]:
# Load the first 100 rows
ca_data = pd.read_csv("../../trade_demo/datasets/ca - feb 2021.csv")[:100]

In [34]:
# We will transfer data in batches of 10

In [39]:
start, end = 0, len(ca_data)
batch_size = 10
idx = 0
while(start < end):
    idx += 1
    
    batch = ca_data[start:start+batch_size] # Select a batch of images
    
    name = f"ca_batch_{idx+1}"  # Asset key name
    dataset_pointer.add(name=name, value=batch)  # Add asset to the dataset pointer
    start += batch_size
    print(f"Batches successfully uploaded: {idx}/{end//batch_size}")



WARNING - Non-DP Asset: You just passed in a asset 'ca_batch_2' which cannot be tracked with differential privacy because it is a <class 'pandas.core.frame.DataFrame'> object.

This means you'll need to manually approve any requests which leverage this data. If this is ok with you, proceed. If you'd like to use automatic differential privacy budgeting, please pass in a DP-compatible tensor type such as by calling .private() on a sy.Tensor with a np.int32 or np.float32 inside.
Are you sure you want to proceed? (y/n)y
Loading dataset... uploading... SUCCESS!Batches successfully uploaded: 1/10


WARNING - Non-DP Asset: You just passed in a asset 'ca_batch_3' which cannot be tracked with differential privacy because it is a <class 'pandas.core.frame.DataFrame'> object.

This means you'll need to manually approve any requests which leverage this data. If this is ok with you, proceed. If you'd like to use automatic differential privacy budgeting, please pass in a DP-compatible tensor type 

In [40]:
# Great !!! we uploaded the whole data into batches of 10.
# Let's check the dataset pointer
dataset_pointer

Dataset: Empty Dataset
Description: Test delete



Asset Key,Type,Shape
"[""ca_batch_2""]",DataFrame,"(10, 22)"
"[""ca_batch_3""]",DataFrame,"(10, 22)"
"[""ca_batch_4""]",DataFrame,"(10, 22)"
"[""ca_batch_5""]",DataFrame,"(10, 22)"
"[""ca_batch_6""]",DataFrame,"(10, 22)"
"[""ca_batch_7""]",DataFrame,"(10, 22)"
"[""ca_batch_8""]",DataFrame,"(10, 22)"
"[""ca_batch_9""]",DataFrame,"(10, 22)"
"[""ca_batch_10""]",DataFrame,"(10, 22)"
"[""ca_batch_11""]",DataFrame,"(10, 22)"


In [41]:
# Let's check the dataset in the domain
domain.datasets

Idx,Name,Description,Assets,Id
[0],Empty Dataset,Test delete,"[""ca_batch_2""] -> DataFrame[""ca_batch_3""] -> DataFrame[""ca_batch_4""] -> DataFrame[""ca_batch_5""] -> DataFrame[""ca_batch_6""] -> DataFrame[""ca_batch_7""] -> DataFrame[""ca_batch_8""] -> DataFrame[""ca_batch_9""] -> DataFrame[""ca_batch_10""] -> DataFrame[""ca_batch_11""] -> DataFrame",17030881-6f2a-45e0-8283-434fdd014740


In [47]:
# Now, lastly, accessing the assets as an iterator
# We want to access the assets of this dataset as an iterator

dataset_ptr = domain.datasets[0] # Selected the MedNIST dataset

# We want an iterator and exclude certain assets.
data_iterator = dataset_ptr.iter(exclude=["ca_batch_2", "ca_batch_3", "ca_batch_4"])

In [48]:
cnt = 0
for d in data_iterator:
    cnt += 1
    print(d)
print(f"total assets in the iterator: {cnt}")

<DataFramePointer -> mednode:fa5193365dd6433996c6e9d3d2440154>
<DataFramePointer -> mednode:a909f5c3392345869c3e142d95464efd>
<DataFramePointer -> mednode:6104f67f1145431991ce4afbc4218146>
<DataFramePointer -> mednode:d023af1ca3fa4971ada4ce1fa244d2e1>
<DataFramePointer -> mednode:eb83e8b89a8b4cbcb8532b018f292337>
<DataFramePointer -> mednode:58dcb428f81148c7a402bc09a0c13dd4>
<DataFramePointer -> mednode:d03fcdcd9a7143c4aa0a176f8580031c>
total assets in the iterator: 7


Since we excluded three assets, so we had pointers to the remaining seven assets.